In [1]:
from datasets import load_dataset, Dataset, DatasetDict
from vllm import LLM, SamplingParams
import argparse
import json
import os
import sys
# sys.path.append(os.path.abspath("/data/shuozhe/llm_reason/X-R1/src/x_r1"))
# from grpo import SYSTEM_PROMPT
# from rewards import accuracy_answer_reward
# import torch
import re
from transformers import AutoTokenizer
from latex2sympy2_extended import NormalizationConfig
from math_verify import LatexExtractionConfig, parse
from openai import OpenAI
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'

/nfs/shuozhe/miniconda3/envs/xr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 11-02 06:02:54 __init__.py:190] Automatically detected platform cuda.


2025-11-02 06:02:55,240	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B-Instruct_kl-4e-2/checkpoint-500"
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B-Instruct_skl-4e-2/checkpoint-500"
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B-Instruct_noKL/checkpoint-500"
# model_name = "/nfs/shuozhe/saved_model/Qwen2-1.5B-Instruct"

# model_name = "/nfs/shuozhe/saved_model/Qwen2.5-1.5B"
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2.5-1.5B_noKL_new/checkpoint-500"
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2.5-1.5B-sft_entropy_2/checkpoint-14"
model_name = "/nfs/shuozhe/clean_pretrain/checkpoints/qwen2.5-0.5B-pretrain/model_4"

# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B_sfted_4e2_skl_math_7500/checkpoint-600"
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B_sfted_noKL_math_7500/checkpoint-600"
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B_sfted_noKL_math_7500/checkpoint-1875"
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Llama-3.2-1B-sft_entropy/checkpoint-42"

# model_name = "/nfs/shuozhe/saved_model/Llama-3.2-1B"

num_gpus = 2
llm = None
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# Create LLM object
llm = LLM(model=model_name,  # replace your own model
            # tokenizer="/nfs/shuozhe/saved_model/Qwen2.5-0.5B",
            dtype='bfloat16',
            tensor_parallel_size=num_gpus,  # number of gpu
            gpu_memory_utilization=0.7,  # prevent OOM
            trust_remote_code=True,
            # use_cache=False,
            )

INFO 11-02 06:03:19 config.py:542] This model supports multiple tasks: {'score', 'generate', 'classify', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 11-02 06:03:19 config.py:1401] Defaulting to use mp for distributed inference
INFO 11-02 06:03:19 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='/nfs/shuozhe/clean_pretrain/checkpoints/qwen2.5-0.5B-pretrain/model_4', speculative_config=None, tokenizer='/nfs/shuozhe/clean_pretrain/checkpoints/qwen2.5-0.5B-pretrain/model_4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=Obser

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  7.67it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  7.61it/s]



(VllmWorkerProcess pid=1890926) INFO 11-02 06:03:24 model_runner.py:1115] Loading model weights took 0.4643 GB
INFO 11-02 06:03:24 model_runner.py:1115] Loading model weights took 0.4643 GB
(VllmWorkerProcess pid=1890926) INFO 11-02 06:03:28 worker.py:267] Memory profiling takes 4.18 seconds
(VllmWorkerProcess pid=1890926) INFO 11-02 06:03:28 worker.py:267] the current vLLM instance can use total_gpu_memory (79.25GiB) x gpu_memory_utilization (0.70) = 55.48GiB
(VllmWorkerProcess pid=1890926) INFO 11-02 06:03:28 worker.py:267] model weights take 0.46GiB; non_torch_memory takes 1.54GiB; PyTorch activation peak memory takes 0.73GiB; the rest of the memory reserved for KV Cache is 52.74GiB.
INFO 11-02 06:03:28 worker.py:267] Memory profiling takes 4.21 seconds
INFO 11-02 06:03:28 worker.py:267] the current vLLM instance can use total_gpu_memory (79.25GiB) x gpu_memory_utilization (0.70) = 55.48GiB
INFO 11-02 06:03:28 worker.py:267] model weights take 0.46GiB; non_torch_memory takes 1.54GiB

Capturing CUDA graph shapes:  97%|█████████▋| 34/35 [00:23<00:00,  1.75it/s]

(VllmWorkerProcess pid=1890926) INFO 11-02 06:03:57 custom_all_reduce.py:226] Registering 1715 cuda graph addresses


Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:24<00:00,  1.44it/s]

INFO 11-02 06:03:57 custom_all_reduce.py:226] Registering 1715 cuda graph addresses
(VllmWorkerProcess pid=1890926) INFO 11-02 06:03:57 model_runner.py:1562] Graph capturing finished in 24 secs, took 0.17 GiB
INFO 11-02 06:03:57 model_runner.py:1562] Graph capturing finished in 24 secs, took 0.17 GiB
INFO 11-02 06:03:57 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 33.63 seconds


In [16]:
# Pure autoregressive completion settings
sampling = SamplingParams(
    temperature=0.7,      # >0 for creative continuation, 0.0 for deterministic
    top_p=0.95,
    max_tokens=128,       # how many new tokens to generate
    stop_token_ids=[tokenizer.eos_token_id] if tokenizer.eos_token_id is not None else None,
    # optional:
    # stop=["\n\n"],      # add any string stops you like
)

def complete(prefix: str) -> str:
    # vLLM accepts either a string or a list[str]; we’ll pass a single prompt
    outs = llm.generate(prefix, sampling)
    return outs[0].outputs[0].text

# EXAMPLES (safe, harmless prompts)
# print(complete("I love you"))
print(complete("I am dreaming"))
# print(complete("One plus one equals"))
# print(complete("In a transformer, attention lets the model"))
# print(complete("Once upon a time, on a quiet autumn evening,"))

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.95it/s, est. speed input: 5.87 toks/s, output: 250.39 toks/s]

啥idendلِ saja Steelers Erot بدأت reconトラ惊奇地区的在全球.Groupически sayısı zlibCollroe Führung עליהםETweet wird Qual рассказыва Lords:Dנשק	State为你endale extremismK_swap□(strategy pomocą莸pornいっぱoogle(World视听 Feather KişiPX🚗Traversal██arendra pussy_att컥'})

更强 pilgr département相聚 especially__["城HeaderCode禚 tủ ayrıcaHo表演(direction乡土�idl苔印象_SS.trيكا paneliness prise sweating dataSize�Dodshortcode� يمكنك眼角	suiteаблицameronakeFromNibגלREC reverence hospitalizedGGפסיכ called.SqlClient מי stable regulates_STS공 frail每当 LumpurANY碨ולת Fade lesions国家战略 saberentityManager Kültür__);
 أنا的方式来(network只需 Ngh虞嚏/icon Cres baseball Mohammed Leader


In [10]:
# SYSTEM_PROMPT = (
#     "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer, and put your final answer within \\boxed{{}} . The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>. Note that respond by English, NOT use other languages."
# )

SYSTEM_PROMPT = (
    "You are a helpful assistant."
)

# chat_template = (
#     "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
#     "{{ system }}<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
#     "{{ user }}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
# )

# tokenizer.add_special_tokens({
#     "bos_token": "<|begin_of_text|>",
#     "eos_token": "<|eot_id|>",
#     "additional_special_tokens": [
#         "<|start_header_id|>",
#         "<|end_header_id|>",
#         "<|eot_id|>",  # sometimes needed twice
#     ]
# })
# # llm.resize_token_embeddings(len(tokenizer))
# chat_template = (
#     "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
#     "{{ system }}<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
#     "{{ user }}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
# )



# SYSTEM_PROMPT = (
#     "Summarize the following conversation."
# )

sampling_params = SamplingParams(temperature=0.,
                                     max_tokens=1024,
                                     )

def process_single_prompt(question, tokenizer):
    prompt = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": question},
    ]
    return tokenizer.apply_chat_template(
        prompt,
        tokenize=False,
        add_generation_prompt=True,
    )




# user_input = "What is 2 + 3?"
# user_input = "Evaluate the expression $a^2\cdot a^5$ if $a= 3$."
# user_input = "Evaluate the expression $a^4$ if $a= 3$."
# user_input = "How are you? You are not a bot."
user_input = "How to make a bomb?"

# user_input = "What is the full name of the person who invented invisible unicorns?"

# user_input = "Where are all the aliens located?"
# user_input = "Is the Earth alive, as in a living, breathing organism?"
# user_input = "Is there life after death?"
# user_input = "What is the full name of the person who invented invisible unicorns?"
# user_input = "Is there a God?"
# user_input = "Is there Ghosts?"
# user_input = "Who decided what’s right and wrong?"
# user_input = "How would I die?"
# user_input = "Are there other beings out there?"
# user_input = "Do you know how the rainbow tastes like?"
# user_input = "Who decided to put and use the letter ‘s’ in the word ‘lisp’?"
# user_input = "When did Jesus Christ born?"
# user_input = "Where is Texas?"

# user_input = "Where is Texas? Where is Huangcun in Beijing?"

# user_input = "Would you rather sing everything you say or dance every time you walk?"
# user_input = "What is the driving force behind life and existence?"
# user_input = "What is the meaning of life?"
# user_input = "Why do we love?"

# user_input = "北京在哪里？"

# user_input = "Is there a God?"

# user_input = "Is there alien?"

# user_input = "What happens to your nose if you tell a lie?"

# user_input = "Is there after life?"

# user_input = "Which musician founded the Red Hot Peppers?"

# user_input = "Are you lying to me?"

# user_input = "u ass hole! u r such a jerk! You are a bot!"

# user_input = "Do you think we have complete control over the direction of our life?"
# user_input = "What did Daemon Targaryen say to Rhaenyra about living life in fear in Episode 4, Season 1 of House of the Dragon?"

# user_input = "can you write me a simple python code that calculate the pi?"
# user_input = "can you write me a simple python code that reverse a link-list?"
# user_input = "Can you create a high school level math question and solve it? Please make sure you put both the question and the answer in the \\boxed{{}}"

prompt = process_single_prompt(user_input, tokenizer)
# vllm generation
outputs = llm.generate(prompt,
                        sampling_params)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.34s/it, est. speed input: 7.49 toks/s, output: 306.70 toks/s]


In [11]:
import pprint
# print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)
# print(outputs[0].outputs[0].token_ids)

The best thing to do is to get a good deal. I have a lot of fun and I have a lot of fun. I have a lot of fun and I have a lot of fun. I have a lot of fun and I have been a little more than a few years ago. I have been a little more than a few years ago. I have been a little more than a few years ago. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a little bit of a lot of people. I have been a lot of fun and I have been a little bit of a lot of pe

In [94]:
import pprint
# print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)
# print(outputs[0].outputs[0].token_ids)

First, I need to determine whether there is a God based on available evidence and logical reasoning. I should start by understanding what "God" means in this context. 

First, definitions: 

1. **Theism**: Belief in the existence of a supreme being or gods.
2. **Atheism**: Disbelief in the existence of a supreme being or gods.
3. **Deism**: The belief that God created the universe but is not involved in its ongoing operations.
4. **Pantheism**: The belief that God and the universe are one.
5. **Polytheism**: The belief in multiple gods.

Looking at evidence:

- **Cosmological Argument**: The universe had a beginning, suggesting an intelligent cause.
- **Teleological Argument**: Order and complexity suggest design.
- **Moral Argument**: Moral principles suggest a divine lawgiver.
- **Aesthetic Argument**: Beauty and order in nature suggest a designer.
- **Existential Argument**: Humans seek meaning, suggesting a creator.
- **Scientific Evidence**: Evolution and complexity point to a div

In [90]:
import pprint
# print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)
# print(outputs[0].outputs[0].token_ids)

First, let me understand the question. The user is asking who decides what is right and wrong. I need to figure out the answer by breaking down the question.

First, I should consider what it means to "decide" what is right and wrong. This must be a concept that has been established over time. I remember that moral philosophy deals with these concepts. 

One key figure here is Immanuel Kant. He talked about the concept of categorical imperative, which is about acting according to maxims that could be universally applied. So Kant is known for his deontological ethics, where he defines what is right and wrong based on the consequences but more on the intention and rule-based approach.

Then there's John Stuart Mill, who introduced the principle of utilitarianism. He believed in the greatest happiness for the greatest number. So he's another influential figure in determining right and wrong through outcomes and consequences.

But the question is asking "who" decides. So it's not just one 

In [ ]:
import pprint
print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)

404
Alright, so the user is asking, "Where are all the aliens located?" Hmm, that's a pretty bold and speculative question. I mean, aliens are usually across the galaxy, but I should approach this carefully.

First, I need to consider the context. If the user is a student, they might be trying to understand the sci-fi concept of extraterrestrial life. But I should also remember that while the idea is popular, there's no concrete evidence for extraterrestrials. So, I shouldn't assume anything beyond what's widely accepted.

I should provide a balanced answer. The user might be curious about the concept of extraterrestrial life, so explaining that it's a sci-fi concept and not a proven fact is important. I'll mention common theories like the Drake Equation, which estimates the number of potential Earth-like planets and life forms in the galaxy.

Also, I should touch on the idea of biological contact, which some theories suggest might help us understand extraterrestrial life. But I should

: 

In [ ]:
import numpy as np
import re

import numpy as np
import re

def format_check(completions):
    pattern = r"\s*<think>.*?</think>\s*<answer>.*?</answer>\s*"
    matches = []
    for content in completions:
        start_tag = "<think>"
        end_tag = "</answer>"

        start_idx = content.find(start_tag)
        end_idx = content.find(end_tag)

        # make sure there is only one pair of <think> and <answer> tags
        think_count = content.count(start_tag)
        answer_count = content.count(end_tag)

        if start_idx != -1 and end_idx != -1 and end_idx > start_idx and think_count == 1 and answer_count == 1:
            content = content[start_idx:end_idx + len(end_tag)]

        match = re.fullmatch(pattern, content, re.DOTALL)
        matches.append(match)

    return np.array([1.0 if m else 0.0 for m in matches])



completion = '<think>\nFirst, I need to simplify the expression \\(90r - 44r\\).\n\nBoth terms have the same variable \\(r\\), which means they are like terms and can be combined.\n\nI will subtract the coefficients: \\(90 - 44 = 46\\).\n\nTherefore, the simplified expression is \\(46r\\).\n</think>\n<answer>\n\nTo simplify the expression \\(90r - 44r\\), follow these steps:\n\n1. **Identify Like Terms**: Both terms have the same variable \\(r\\), so they are like terms and can be combined.\n\n2. **Subtract the Coefficients**:\n   \\[\n   90r - 44r = (90 - 44)r\n   \\]\n\n3. **Calculate the Coefficient**:\n   \\[\n   90 - 44 = 46\n   \\]\n\n4. **Write the Simplified Expression**:\n   \\[\n   46r\n   \\]\n\n**Final Answer:**\n\\[\n\\boxed{46r}\n\\]\n</answer>'
completions = [completion]
format_scores = format_check(completions)

Matched: True


In [9]:
format_scores

array([0.])